In [1]:
%load_ext autoreload
%autoreload 2
#from src.VLLMServer import launch_command,get_client_dict,ask_llm,ask_llm_prompt
from src.code_utils import extract_code_block
from src.process_openmath_ja import parse_record,eval_answer
import yaml
from vllm import SamplingParams, LLM
import json

/home/setup/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-20 11:34:53,104	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
out_path="processed/data.jsonl"

In [3]:

#load dataset
from datasets import load_dataset
ds=load_dataset("kunishou/OpenMathInstruct-1-1.8m-ja",split="train",streaming=False)

#ds=ds.shuffle()

In [4]:
model_name = "microsoft/Phi-3-medium-128k-instruct"

#llm = LLM(model=model_name, trust_remote_code=True,
#          max_model_len=20000
#          )
model_name = "karakuri-ai/karakuri-lm-8x7b-chat-v0.1"
llm = LLM(model=model_name, trust_remote_code=True,
          max_model_len=20000, tensor_parallel_size=2,
          )



/home/setup/miniconda3/envs/llmeval/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-06-20 11:35:15,615	INFO worker.py:1749 -- Started a local Ray instance.


INFO 06-20 11:35:16 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='karakuri-ai/karakuri-lm-8x7b-chat-v0.1', speculative_config=None, tokenizer='karakuri-ai/karakuri-lm-8x7b-chat-v0.1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=20000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=karakuri-ai/karakuri-lm-8x7b-chat-v0.1)
INFO 06-20 11:35:19 utils.py:660] Found nccl from library /home/setup/.config/vllm/nccl/cu12/libnccl.so.2.18.1
(RayWorkerWrapper pid=916635) INFO 06-20 11:35:19 utils.py:660] Found nccl from library /home/setup/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 06-20 11:35:20 selector.py:

In [5]:
model_name="microsoft/Phi-3-medium-128k-instruct"
pre_command="""次の問題を解きなさい｡出力事項は以下のとおりである｡
#解説: 問題を解くための基本的な考え方を日本語で出力する｡
#Python: 計算プログラム｡以下の例を参考に､出力形式を守ること｡
```
#a+bの計算を行う
a=1
b=2
print(a+b)
```
#問題
"""

In [6]:
from tqdm import tqdm
batch_size=100
idx=0


for j in tqdm(range(int(len(ds)/batch_size))):

    prompts=[]
    answer_list=[]
    question_list=[]
    idx_list=[]

    for i in range(batch_size):
        record=ds[idx]
        idx+=1
        #問題文と回答の取得
        question,answer=parse_record(record)
        answer_list.append(answer)
        question_list.append(question)
        idx_list.append(idx)

        #promptの生成
        ja_text=pre_command+question
        prompt= f"""<|user|>
    {ja_text}<|end|>
    <|assistant|>"""
        prompts.append(prompt)


    #回答の生成
    outputs = llm.generate(
        prompts,
        sampling_params=SamplingParams(
            temperature=0.1,
            max_tokens=2048,
            repetition_penalty=1.2,
        )
    )

    #回答のチェック
    for i in range(len(outputs)):
        res=outputs[i].outputs[0].text.strip()
        res=res.replace("\'","`")

        #生成されたコードを実際にまわして評価する
        explanation_block=res[:res.find("```")]
        try:
            code_block=extract_code_block(res)
        except:
            continue


        answer=answer_list[i]
        question=question_list[i]
        #真偽判定
        is_correct_answer=eval_answer(code_block,answer,verbose=False)

        generated_prompt=explanation_block+"\n"+code_block
        if is_correct_answer:
            new_record={
                    "id":idx_list[i],
                    "question":question,
                    "prediction":generated_prompt,
                    "answer":answer,

            }
            with open(out_path,"a") as f:
                f.write(json.dumps(new_record,ensure_ascii=False)+"\n")

  0%|          | 0/18250 [00:00<?, ?it/s]

  0%|          | 1/18250 [01:25<432:26:40, 85.31s/it]


KeyboardInterrupt: 

In [8]:
answer_list[1]

'756'

In [9]:
res

'##解答：\nまず、スティーブが持つ全体の数（x）を求めます。それぞれの人物がどれだけ所有しているかわからない状態でも可能です。\n- x = 3 * (スティーブの量 / スティーブの係数)\u3000→\xa0x = 3 * (84/10) → x = 25.2\n\n上記より「全員合わせて」約25個の結果が得られます。続いて各自の保有率を掛け算します。\n- Mitchilda : ½, Matt : 1 , Steve : 10  ※Steve`s amount is already known as "x" above and we can use it here without recomputing again! :)\n- Mitchilda : ¼ of the total -> y = .25*y + z where `z` will be computed next for Matt below...\n- Matt      : half of what he has now - that means subtract from his current value which was just calculated in step #1 to get a new variable named Z --> z=(x-.76)/2 or simply put into one line with previous equation like so; y = .25*(x-(.75))/(2)=.19..etcetera.........infinitum ad nauseam if desired but let’s stop there since this isn’t an infinite series problem after all even though technically speaking mathematically speaking theoretically speaking hypothetically speaking conjecturally speaking speculatively speaking possibly maybe perhaps potentially probably improbabl